In [1]:
import os
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import dask.dataframe as dd

In [2]:
df = pd.read_csv("../input/harddrive.csv")
df.head()

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2016-01-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,135.0,108.0,143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01,Z305B2QN,ST4000DM000,1.976651e-311,0,113,54551400,NaN,NaN,96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,136.0,104.0,124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,136.0,104.0,137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01,WD-WMC4N2899475,WDC WD30EFRX,1.482490e-311,0,200,0,NaN,NaN,175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(3179295, 95)

In [4]:
df = df[["date", "serial_number", "model", "failure", "capacity_bytes",
                  "smart_2_normalized",# "smart_2_raw",
                 "smart_5_normalized", #"smart_5_raw",
                 "smart_9_normalized", #"smart_9_raw",
                 "smart_187_normalized",# "smart_187_raw",
                 "smart_188_normalized",# "smart_188_raw",
                 "smart_192_normalized",# "smart_192_raw",
                 "smart_194_normalized",# "smart_194_raw",
                 "smart_197_normalized",# "smart_197_raw",
                 "smart_198_normalized",]]# "smart_198_raw",]]

In [5]:
df.isnull().sum()

date                          0
serial_number                 0
model                         0
failure                       0
capacity_bytes                0
smart_2_normalized      1920177
smart_5_normalized            0
smart_9_normalized            0
smart_187_normalized    1372779
smart_188_normalized    1372779
smart_192_normalized       9581
smart_194_normalized         52
smart_197_normalized          0
smart_198_normalized          0
dtype: int64

In [6]:
df['capacity_bytes'] = df['capacity_bytes']/(1024*4)
df.head()

,date,serial_number,model,failure,capacity_bytes,smart_2_normalized,smart_5_normalized,smart_9_normalized,smart_187_normalized,smart_188_normalized,smart_192_normalized,smart_194_normalized,smart_197_normalized,smart_198_normalized
0,2016-01-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,0,3.619360e-315,135.0,100,97,NaN,NaN,100.0,253.0,100,100
1,2016-01-01,Z305B2QN,ST4000DM000,0,4.825809e-315,NaN,100,100,100.0,100.0,100.0,30.0,100,100
2,2016-01-01,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,0,3.619360e-315,136.0,100,97,NaN,NaN,100.0,250.0,100,100
3,2016-01-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,0,3.619360e-315,136.0,100,97,NaN,NaN,100.0,193.0,100,100
4,2016-01-01,WD-WMC4N2899475,WDC WD30EFRX,0,3.619360e-315,NaN,200,84,NaN,NaN,200.0,121.0,200,100


In [7]:
df['Brands'] = " "
df['Brands'][((df['model'].str.contains("ST")==True) & (df['model'].str.contains("HGST")==False))] = "Seagate"
df['Brands'][df['model'].str.contains("HGST")] = "HGST"
df['Brands'][df['model'].str.contains("Hitachi")] = "Hitachi"
df['Brands'][df['model'].str.contains("Toshiba" or "TOSHIBA")] = "Toshiba"
df['Brands'][df['model'].str.contains("WDC")] = "Western Digital"
df['Brands'][df['model'].str.contains("SAMSUNG")] = "Samsung"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ip

In [8]:
df['Brands'][df['Brands'] == " "] = "Toshiba"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df.head()

,date,serial_number,model,failure,capacity_bytes,smart_2_normalized,smart_5_normalized,smart_9_normalized,smart_187_normalized,smart_188_normalized,smart_192_normalized,smart_194_normalized,smart_197_normalized,smart_198_normalized,Brands
0,2016-01-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,0,3.619360e-315,135.0,100,97,NaN,NaN,100.0,253.0,100,100,Hitachi
1,2016-01-01,Z305B2QN,ST4000DM000,0,4.825809e-315,NaN,100,100,100.0,100.0,100.0,30.0,100,100,Seagate
2,2016-01-01,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,0,3.619360e-315,136.0,100,97,NaN,NaN,100.0,250.0,100,100,Hitachi
3,2016-01-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,0,3.619360e-315,136.0,100,97,NaN,NaN,100.0,193.0,100,100,Hitachi
4,2016-01-01,WD-WMC4N2899475,WDC WD30EFRX,0,3.619360e-315,NaN,200,84,NaN,NaN,200.0,121.0,200,100,Western Digital


In [10]:
df_total = df.groupby('Brands')['failure'].count().sort_values(ascending=False)

In [11]:
pd.DataFrame(df_total)
df_total.head()

Brands
Seagate            1839553
Hitachi             650734
HGST                562944
Western Digital     113713
Toshiba              12345
Name: failure, dtype: int64

In [12]:
#Failure count for each manufacturer
df.groupby('Brands')['failure'].sum().sort_values()

Brands
Samsung              0
Toshiba              2
HGST                 3
Hitachi             22
Western Digital     26
Seagate            162
Name: failure, dtype: int64

In [13]:
#Failure count for each model
df.groupby('model')['failure'].sum().sort_values(ascending=False)

model
ST4000DM000                139
ST320LT007                  15
Hitachi HDS722020ALA330     13
WDC WD800AAJS                6
WDC WD30EFRX                 6
Hitachi HDS5C3030ALA630      5
Hitachi HDS5C4040ALE630      4
WDC WD20EFRX                 3
ST3160318AS                  2
ST4000DX000                  2
WDC WD10EADS                 2
HGST HMS5C4040ALE640         2
WDC WD1600AAJS               2
WDC WD1600AAJB               2
ST6000DX000                  2
ST3160316AS                  1
TOSHIBA MD04ABA500V          1
WDC WD3200BEKT               1
TOSHIBA DT01ACA300           1
WDC WD3200BEKX               1
ST9250315AS                  1
HGST HMS5C4040BLE640         1
WDC WD60EFRX                 1
WDC WD800AAJB                1
WDC WD800BB                  1
ST3500320AS                  0
ST31500341AS                 0
ST31500541AS                 0
WDC WD800LB                  0
ST250LT007                   0
                          ... 
HGST HUH728080ALE600         0
HG

In [14]:
#Total null in each column grouped by brands
label_cols = [   "smart_2_normalized",# "smart_2_raw",
                 "smart_5_normalized",# "smart_5_raw",
                 "smart_9_normalized",# "smart_9_raw",
                 "smart_187_normalized",# "smart_187_raw",
                 "smart_188_normalized",# "smart_188_raw",
                 "smart_192_normalized",# "smart_192_raw",
                 "smart_194_normalized",# "smart_194_raw",
                 "smart_197_normalized",# "smart_197_raw",
                 "smart_198_normalized",]# "smart_198_raw",]
df_nulls = []
for col in label_cols:
    #print(col + ":")
    df_nulls.append(df[col].isnull().groupby([df['Brands']]).sum())
    
    #print("\n")

In [15]:
df_total = pd.DataFrame(df_total)
df_total.columns = ['Total']
df_total

,Total
Brands,
Seagate,1839553
Hitachi,650734
HGST,562944
Western Digital,113713
Toshiba,12345
Samsung,6


In [16]:
df_total.loc['HGST']

Total    562944
Name: HGST, dtype: int64

In [17]:
df_nulls = pd.DataFrame(df_nulls)
df_nulls

Brands,HGST,Hitachi,Samsung,Seagate,Toshiba,Western Digital
smart_2_normalized,0.0,0.0,6.0,1806458.0,0.0,113713.0
smart_5_normalized,0.0,0.0,0.0,0.0,0.0,0.0
smart_9_normalized,0.0,0.0,0.0,0.0,0.0,0.0
smart_187_normalized,562944.0,650734.0,0.0,33095.0,12345.0,113661.0
smart_188_normalized,562944.0,650734.0,0.0,33095.0,12345.0,113661.0
smart_192_normalized,0.0,0.0,6.0,9275.0,0.0,300.0
smart_194_normalized,0.0,0.0,0.0,0.0,0.0,52.0
smart_197_normalized,0.0,0.0,0.0,0.0,0.0,0.0
smart_198_normalized,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_nulls = df_nulls.transpose()
df_nulls

,smart_2_normalized,smart_5_normalized,smart_9_normalized,smart_187_normalized,smart_188_normalized,smart_192_normalized,smart_194_normalized,smart_197_normalized,smart_198_normalized
Brands,,,,,,,,,
HGST,0.0,0.0,0.0,562944.0,562944.0,0.0,0.0,0.0,0.0
Hitachi,0.0,0.0,0.0,650734.0,650734.0,0.0,0.0,0.0,0.0
Samsung,6.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
Seagate,1806458.0,0.0,0.0,33095.0,33095.0,9275.0,0.0,0.0,0.0
Toshiba,0.0,0.0,0.0,12345.0,12345.0,0.0,0.0,0.0,0.0
Western Digital,113713.0,0.0,0.0,113661.0,113661.0,300.0,52.0,0.0,0.0


In [19]:
from sklearn.preprocessing import Imputer

In [20]:
for col in label_cols:
    #print(col)
    print("\n")
    for brand in df_nulls.index:
        #print(str(col) + ":" + str(df_nulls[col].loc[brand]))
        nulls = df_nulls[col].loc[brand]
        total = df_total['Total'].loc[brand]
        #print("Total:", total)
        null_perc = float(nulls/total)*100
        if null_perc > 50:#df_nulls[col].loc[brand] == total:
            print(str(col) + " -> " +  str(brand) + " -> " + "Not recorded")
            #df[col] = [df[col].fillna('NR') for i in df['Brands'] == brand]
            #df[col] = df[col].apply(lambda x: x.fillna('NR') for i in df['Brands'] == brand)
            not_recorded_index = df[df.loc[:, "Brands"] == brand].index
            df.loc[not_recorded_index, col] = 9999
        #else:
         #   imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
         #   null_brands       = df[df.loc[:, "Brands"] == brand]
         #   null_brands_index = df[df.loc[:, "Brands"] == brand].index
         #   print(brand)
         #   print(null_brands_index)
         #   imp = imp.fit(null_brands[label_cols])
         #   df.loc[null_brands_index, col] = imp.transform(df[label_cols])



smart_2_normalized -> Samsung -> Not recorded
smart_2_normalized -> Seagate -> Not recorded
smart_2_normalized -> Western Digital -> Not recorded






smart_187_normalized -> HGST -> Not recorded
smart_187_normalized -> Hitachi -> Not recorded
smart_187_normalized -> Toshiba -> Not recorded
smart_187_normalized -> Western Digital -> Not recorded


smart_188_normalized -> HGST -> Not recorded
smart_188_normalized -> Hitachi -> Not recorded
smart_188_normalized -> Toshiba -> Not recorded
smart_188_normalized -> Western Digital -> Not recorded


smart_192_normalized -> Samsung -> Not recorded








In [21]:
df.isnull().sum()

date                        0
serial_number               0
model                       0
failure                     0
capacity_bytes              0
smart_2_normalized          0
smart_5_normalized          0
smart_9_normalized          0
smart_187_normalized    33095
smart_188_normalized    33095
smart_192_normalized     9575
smart_194_normalized       52
smart_197_normalized        0
smart_198_normalized        0
Brands                      0
dtype: int64

In [ ]:
"""df = df[[        "smart_5_normalized",# "smart_5_raw",
                 "smart_9_normalized",# "smart_9_raw",
                 "smart_187_normalized",# "smart_187_raw",
                 "smart_188_normalized",# "smart_188_raw",
                 "smart_192_normalized",# "smart_192_raw",
                 "smart_194_normalized",# "smart_194_raw",
                 "smart_197_normalized",# "smart_197_raw",
                 "smart_198_normalized"]]# "smart_198_raw",]"""

In [ ]:
"""df_hgst = df[df['Brands'] == 'HGST']
df_hitachi = df[df['Brands'] == 'Hitachi']
df_samsumg = df[df['Brands'] == 'Samsung']
df_seagate = df[df['Brands'] == 'Seagate']
df_toshiba = df[df['Brands'] == 'Toshiba']
df_wd = df[df['Brands'] == 'Western Digital']"""

In [ ]:
"""df_hgst.reset_index(inplace=True)
df_hitachi.reset_index(inplace=True)
df_samsumg.reset_index(inplace=True)
df_seagate.reset_index(inplace=True)
df_toshiba.reset_index(inplace=True)
df_wd.reset_index(inplace=True)"""

In [ ]:
#df_hgst.head()

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
df_temp = df

In [25]:
"""imp = Imputer(missing_values='NaN', strategy='mean', axis=0)

for col in label_cols:
    for brand in df_nulls.index:
        brand_total       = df[df.loc[:, "Brands"] == brand]
        brand_total_index = df[df.loc[:, "Brands"] == brand].index
        brand_notnull     = brand_total[brand_total[col].isnull() == False]
        print("Not nullshape: ", brand_notnull.shape)
        #brand_notnull     = brand_notnull[col] 
        
        brand_nulls       = brand_total[brand_total[col].isnull()]
        print("Null shape: ", brand_nulls.shape)
        #brand_nulls       = brand_nulls[col]
        
        #mode = brand_total.filter(cols).mode()
        #print("Mode: ", mode)
        
        #df.loc[brand_total_index, col] = df.loc[brand_total_index, col].fillna(brand_notnull[col].mode())
        if brand_nulls.shape[0] != 0:
            #X_train, X_test, y_train, y_test = train_test_split(brand_total[label_cols], brand_total[col], test_size=0.30)
            imp.fit(brand_total[[col]])
            df_temp.loc[brand_total_index, col] = imp.transform(df_temp.loc[[brand_total_index, col]]).ravel()
            print(df_temp[col])
        #y_test[col] = imp.predict(X_test)
            #print(accuracy_score(y_test, X_test[col]))"""

Not nullshape:  (562944, 15)
Null shape:  (0, 15)
Not nullshape:  (650734, 15)
Null shape:  (0, 15)
Not nullshape:  (6, 15)
Null shape:  (0, 15)
Not nullshape:  (1839553, 15)
Null shape:  (0, 15)
Not nullshape:  (12345, 15)
Null shape:  (0, 15)
Not nullshape:  (113713, 15)
Null shape:  (0, 15)
Not nullshape:  (562944, 15)
Null shape:  (0, 15)
Not nullshape:  (650734, 15)
Null shape:  (0, 15)
Not nullshape:  (6, 15)
Null shape:  (0, 15)
Not nullshape:  (1839553, 15)
Null shape:  (0, 15)
Not nullshape:  (12345, 15)
Null shape:  (0, 15)
Not nullshape:  (113713, 15)
Null shape:  (0, 15)
Not nullshape:  (562944, 15)
Null shape:  (0, 15)
Not nullshape:  (650734, 15)
Null shape:  (0, 15)
Not nullshape:  (6, 15)
Null shape:  (0, 15)
Not nullshape:  (1839553, 15)
Null shape:  (0, 15)
Not nullshape:  (12345, 15)
Null shape:  (0, 15)
Not nullshape:  (113713, 15)
Null shape:  (0, 15)
Not nullshape:  (562944, 15)
Null shape:  (0, 15)
Not nullshape:  (650734, 15)
Null shape:  (0, 15)
Not nullshape: 

TypeError: unhashable type: 'Int64Index'

In [26]:
brands = df['Brands'].unique()
brands

array(['Hitachi', 'Seagate', 'Western Digital', 'HGST', 'Toshiba',
       'Samsung'], dtype=object)

In [28]:
for brand in brands:
        brand_total       = df_temp[df_temp.loc[:, "Brands"] == brand]
        brand_total_index = df_temp[df_temp.loc[:, "Brands"] == brand].index
        brand_notnull     = brand_total[brand_total[col].isnull() == False]
        #brand_notnull     = brand_notnull[col] 
        
        brand_nulls       = brand_total[brand_total[col].isnull()]
        #brand_nulls       = brand_nulls[col]
        
        mean = brand_total.filter(label_cols).mean()
        print("Brand: ", brand)
        print("Mean: ", mean)
        
        df_temp.loc[brand_total_index, label_cols] = df_temp.loc[brand_total_index, label_cols].fillna(mean.iloc[0])#df.mode().iloc[0])"""

Brand:  Hitachi
Mean:  smart_2_normalized       118.897792
smart_5_normalized        99.924393
smart_9_normalized        95.399092
smart_187_normalized    9999.000000
smart_188_normalized    9999.000000
smart_192_normalized      99.690274
smart_194_normalized     217.054661
smart_197_normalized     100.000000
smart_198_normalized     100.000000
dtype: float64
Brand:  Seagate
Mean:  smart_2_normalized      9999.000000
smart_5_normalized       102.732026
smart_9_normalized        89.643884
smart_187_normalized      99.965844
smart_188_normalized      99.999995
smart_192_normalized     102.259307
smart_194_normalized      27.813658
smart_197_normalized     102.734089
smart_198_normalized     102.734089
dtype: float64
Brand:  Western Digital
Mean:  smart_2_normalized      9999.000000
smart_5_normalized       199.997230
smart_9_normalized        79.585017
smart_187_normalized    9999.000000
smart_188_normalized    9999.000000
smart_192_normalized     199.999947
smart_194_normalized     121.

In [29]:
print(df_temp.isnull().sum())

date                    0
serial_number           0
model                   0
failure                 0
capacity_bytes          0
smart_2_normalized      0
smart_5_normalized      0
smart_9_normalized      0
smart_187_normalized    0
smart_188_normalized    0
smart_192_normalized    0
smart_194_normalized    0
smart_197_normalized    0
smart_198_normalized    0
Brands                  0
dtype: int64


In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
clf = RandomForestClassifier(random_state=22)

In [32]:
from sklearn.model_selection import train_test_split

In [43]:
reg_cols = [ #"capacity_bytes", #"failure",
                  "smart_2_normalized",# "smart_2_raw",
                 "smart_5_normalized", #"smart_5_raw",
                 "smart_9_normalized", #"smart_9_raw",
                 "smart_187_normalized",# "smart_187_raw",
                 "smart_188_normalized",# "smart_188_raw",
                 "smart_192_normalized",# "smart_192_raw",
                 "smart_194_normalized",# "smart_194_raw",
                 "smart_197_normalized",# "smart_197_raw",
                 "smart_198_normalized"
]

In [44]:
X_train, X_test, y_train, y_test = train_test_split(df_temp[reg_cols], df['failure'], test_size=0.25, stratify=df['failure'])

In [45]:
y_train.sum()

161

In [46]:
#df[label_cols] = df[label_cols].replace("NR", np.nan)

In [47]:
clf = clf.fit(X_train, y_train)

In [48]:
X_test_pred = clf.predict(X_test)

In [49]:
X_test_pred

array([0, 0, 0, ..., 0, 0, 0])

In [50]:
from sklearn import metrics

In [51]:
print('logloss', metrics.log_loss(y_true=y_test, y_pred=X_test_pred))
print('roc_auc', metrics.roc_auc_score(y_true=y_test, y_score=X_test_pred))

logloss 0.0023465536064785296
roc_auc 0.5370345205857366


In [52]:
from sklearn.metrics import classification_report,confusion_matrix

In [53]:
print(confusion_matrix(y_test,X_test_pred))

[[794766      4]
 [    50      4]]


In [54]:
print(classification_report(y_test,X_test_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    794770
          1       0.50      0.07      0.13        54

avg / total       1.00      1.00      1.00    794824



In [55]:
from sklearn.linear_model import LogisticRegression

In [56]:
clf_log = LogisticRegression()

In [57]:
clf_log = clf_log.fit(X_train, y_train)

In [58]:
X_log_pred = clf_log.predict(X_test)
X_log_pred

array([0, 0, 0, ..., 0, 0, 0])

In [59]:
print('logloss', metrics.log_loss(y_true=y_test, y_pred=X_log_pred))
print('roc_auc', metrics.roc_auc_score(y_true=y_test, y_score=X_log_pred))

logloss 0.0023465495824559527
roc_auc 0.5
